# Classificação de Estágios da Doença de Alzheimer

## Convolutional Neural Network (ResNet)

In [114]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from PIL import Image
from sklearn.calibration import label_binarize
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    cohen_kappa_score,
    confusion_matrix,
    hamming_loss,
    jaccard_score,
    log_loss,
    matthews_corrcoef,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

In [115]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import random_split

### Variáveis iniciais

In [116]:
# Valor fixo de seed para reprodutibilidade
random_seed = 42

# Pasta do dataset
dataset_folder = "../data/Alzheimer_MRI_4_classes_dataset"
folders = ["NonDemented", "VeryMildDemented", "MildDemented", "ModerateDemented"]

# Número máximo de imagens a serem carregadas em cada classe
max_images_per_class = 100

### Random Seed

In [117]:
random.seed(random_seed)
np.random.seed(random_seed)

### Carregar dados

In [118]:
# Dataset customizado de imagens
class AlzheimerDataset(Dataset):
    def __init__(
        self, dataset_folder, folders, transform=None, max_images_per_class=100
    ):
        self.dataset_folder = dataset_folder
        self.folders = folders
        self.transform = transform
        self.max_images_per_class = max_images_per_class
        self.images = []
        self.labels = []
        self.class_to_idx = {folder: idx for idx, folder in enumerate(folders)}

        # Carregar as imagens e os rótulos
        for folder in folders:
            folder_path = os.path.join(dataset_folder, folder)
            if os.path.exists(folder_path):
                count = 0
                for img_name in os.listdir(folder_path):
                    if count >= max_images_per_class:
                        break
                    img_path = os.path.join(folder_path, img_name)
                    try:
                        image = Image.open(img_path).convert("RGB")
                        label = self.class_to_idx[folder]  # Converter para índice
                        if self.transform:
                            image = self.transform(image)
                        self.images.append(image)
                        self.labels.append(label)
                        count += 1
                    except Exception as e:
                        print(f"Erro ao processar a imagem {img_name}: {e}")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Retorna a imagem e o rótulo como tensores
        return self.images[idx], torch.tensor(self.labels[idx])

# Transformação de pré-processamento
transform = transforms.Compose(
    [
        transforms.Resize((176, 128)),  # Redimensionar para 176x128
        transforms.ToTensor(),  # Converte a imagem para tensor
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # Normalização com valores padrão para ResNet
        # transforms.Normalize((0.5,), (0.5,)),  # Normalização
    ]
)

# Criar o dataset
train_dataset = AlzheimerDataset(
    dataset_folder=dataset_folder,
    folders=folders,
    transform=transform,
    max_images_per_class=max_images_per_class,
)

# Proporção do conjunto de validação
val_split_ratio = 0.1

# Cálculo dos tamanhos dos conjuntos
total_size = len(train_dataset)
val_size = int(total_size * val_split_ratio)
train_size = total_size - val_size

# Divide o dataset em treinamento e validação
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Cria os DataLoaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)  # shuffle=False para validação

# Criar DataLoader
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

### Rótulos

In [119]:
# # Codificar os rótulos
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(folders)

# # Criar o dataset e DataLoader
# dataset = AlzheimerDataset(dataset_folder, folders, transform=transform, max_images_per_class=max_images_per_class)

### Dividir em treino e teste

In [120]:
# # Dividir em treinamento e teste
# train_size = int(0.8 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [121]:
# # Verificar o tamanho do DataLoader
# print(f"Train loader size: {len(train_loader.dataset)}")
# print(f"Test loader size: {len(test_loader.dataset)}")

### Treinar o modelo

In [122]:
# Carregar o ResNet50
model = models.resnet50(pretrained=True)

# Ajustar a última camada para o número de classes
model.fc = torch.nn.Linear(model.fc.in_features, len(folders))

In [123]:
# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()  # Usado para problemas de classificação multiclasse
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Otimizador Adam

In [124]:
# Utilizar GPU se disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [125]:
num_epochs = 12

model.train()  # Coloca o modelo no modo de treinamento

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:

        # Mover os dados para o dispositivo (GPU/CPU)
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero os gradientes dos parâmetros do modelo
        optimizer.zero_grad()

        # Passa os dados pelo modelo
        outputs = model(inputs)

        # Calcula a perda
        loss = criterion(outputs, labels)

        # Retropropagação (backpropagation) para atualizar os pesos
        loss.backward()

        # Atualiza os parâmetros
        optimizer.step()

        # Predições
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Acumular a perda para monitoramento
        running_loss += loss.item()

    # Métricas de treinamento
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    # # Avaliação no conjunto de validação
    # model.eval()  # Modo de avaliação
    # val_loss = 0.0
    # val_correct = 0
    # val_total = 0

    # # Desativa gradientes para validação
    # with torch.no_grad():
    #     for val_inputs, val_labels in val_loader:
    #         val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
    #         val_outputs = model(val_inputs)
    #         val_loss += criterion(val_outputs, val_labels).item()
    #         _, val_predicted = torch.max(val_outputs, 1)
    #         val_total += val_labels.size(0)
    #         val_correct += (val_predicted == val_labels).sum().item()

    # val_loss /= len(val_loader)
    # val_accuracy = 100 * val_correct / val_total

    # print(
    #     f"Epoch [{epoch+1}/{num_epochs}], "
    #     f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.2f}%, "
    #     f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%"
    # )

    # Exibir informações sobre o progresso
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

Epoch [1/12], Loss: 0.9986, Accuracy: 57.38%
Epoch [2/12], Loss: 0.5691, Accuracy: 78.30%
Epoch [3/12], Loss: 0.4263, Accuracy: 83.66%
Epoch [4/12], Loss: 0.3314, Accuracy: 86.91%
Epoch [5/12], Loss: 0.1983, Accuracy: 92.09%
Epoch [6/12], Loss: 0.1591, Accuracy: 93.76%
Epoch [7/12], Loss: 0.1525, Accuracy: 94.29%
Epoch [8/12], Loss: 0.1640, Accuracy: 93.94%
Epoch [9/12], Loss: 0.0786, Accuracy: 97.63%
Epoch [10/12], Loss: 0.1016, Accuracy: 96.66%
Epoch [11/12], Loss: 0.1032, Accuracy: 96.13%
Epoch [12/12], Loss: 0.0380, Accuracy: 98.77%
